In [1]:
#Task 17 : LangGraph - day5 
#Implementing Supervisor agent and Swarms

from langchain_ollama import ChatOllama
import re

# -------------------------
#  Base LLM & Tool
# -------------------------

# Connect to llama3
llm = ChatOllama(model="llama3", temperature=0)

# Define tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

# -------------------------
#  Worker Agent
# -------------------------

def worker_agent(task_prompt: str) -> str:
    """
    Worker agent:
    - Invokes LLM to reason
    - Executes multiply if requested
    """
    response = llm.invoke(task_prompt).content
    
    # Parse tool call
    if "Action:" in response:
        numbers = re.findall(r"\d+", response)
        if len(numbers) >= 2:
            result = multiply(int(numbers[0]), int(numbers[1]))
            return str(result)
    return response

# -------------------------
#  Supervisor Agent
# -------------------------

def supervisor_agent(question: str) -> str:
    """
    Supervisor agent:
    - Decides subtasks
    - Sends subtasks to worker swarm
    - Aggregates worker results
    """
    # Step 1: Ask supervisor to split tasks
    prompt = f"""
You are a supervisor agent. Decide if this question needs subtasks.
Question: {question}

Respond in the format:
Subtasks:
1: <task description>
2: <task description>
If only one task, just list 1.
"""
    response = llm.invoke(prompt).content
    print("Supervisor suggested subtasks:")
    print(response)
    
    # Step 2: Extract tasks
    tasks = re.findall(r"\d+: (.+)", response)
    
    # Step 3: Send tasks to workers (swarm)
    worker_results = [worker_agent(task) for task in tasks]
    
    # Step 4: Aggregate
    final_prompt = f"""
You are a supervisor agent.
The original question was: {question}
The worker results are: {worker_results}
Provide the final answer.
"""
    final_response = llm.invoke(final_prompt).content
    return final_response

# -------------------------
#  Run Example
# -------------------------

if __name__ == "__main__":
    question = "What is 7 multiplied by 6?"
    final_answer = supervisor_agent(question)
    print("\nFinal Answer:")
    print(final_answer)


Supervisor suggested subtasks:
No subtasks needed for this question! The answer can be calculated directly:

No subtasks.

Final Answer:
As a supervisor agent, I will review the worker's results and provide the final answer.

Since the worker's results are [], it means they have not provided any calculation or result. Therefore, I will perform the calculation myself to provide the correct answer.

7 multiplied by 6 is:

7 × 6 = 42

So, the final answer is: 42
